# Trabalho 3

### Introdução

Atualmente sabe-se que algoritmos quânticos podem comprometer as técnicas criptográficas convencionais. Assim é necessário recorrer a técnicas criptográficas pós-quânticas que permitam resistir a ataques de computadores quânticos.  
Neste trabalho foi pedida uma implementação KEM-IND-CPA e uma implementação PKE-IND-CCA sobre dois dos algoritmos pós quânticos atualmente viáveis nomeadamente:

- NTRU
- NewHope  
  
  
Explicaremos ambas as implementações pela ordem em que são referidas para cada um dos algoritmos.

### NTRU

#### KEM-IND-CPA

##### KeyGen()

- O primeiro passo é a geração de um elemento $g$ pequeno em R, $g$ tem de ser invertivel em R/3. Existem várias maneiras de garantir isso, nós fizemos uso da função $is_unit()$ sobre R3($g$).
- Em seguida geramos um $f$ pequeno em R com peso $w$. De seguida calculamos a inversa de $g$ á qual chamamos $g_inv$.
- Por fim guardamos a chave secreta $(f , g_inv)$ e calculamos a chave pública  $Rq(g)/Rq(3*f)$ .  
  
  
##### Encapsulate()

- Começamos por calcular um polinómio $r$ pequeno de peso $w$ em R.
- De seguida calculamos um Hash do $r$ ao qual chamamos $key$. E calculamos também o criptograma $C$ composto pela multiplicação $r*pk$ em q, sendo $pk$ a chave pública, seguida de um arredondamento para o múltiplo de 3 mais próximo. 
- Retornamos o par $(key,C)$.  
  
  
##### Decapsulate()

- O primeiro passo é a multiplicação de criptograma por $3*f$ em Rq, guardando o resultado na variavel $pre_process$.
- Em seguida, pondo cada coeficiente do resultado anterior entre −(q − 1)/2 e (q − 1)/2, reduzimos módulo 3 obtendo $e$ em R/3.
- $e$ é igual a $g*r$ em R3, então para obtermos $r'$ basta-nos multiplicar $e$ pela inversa de $g$ aplicando em seguida o $lift()$. 
- Por fim fizemos o Hash do $r'$ e devolvemos comparando o resultado obtido com a chave devolvida no $Encrypt()$ verificando a correta implementação do algoritmo.

#### PKE-IND-CPA
  
  
##### KeyGen()  
  
Esta função manteve-se igual á função impleentada na versão KEM-IND-CPA. 
  
##### Encapsulate()  

- Nesta função começamos por proceder á geração de um polinómio $r$ pequeno de peso $w$ em R. 
- De seguida foi gerado um polinómio $salt$. 
- Calculamos o criptograma como $r*pk$.
- Por fim calculamos a $key$ fazendo um HMAC sobre o $r$ usando o $salt$ como chave e devolvemos o par $(key,c)$.
  

##### Decapsulate()  
 
- Começamos por abrir a chave secreta e obter $(f, g_inv)$.
- Á semelhança do Decapsulate feito no esquema KEM-IND-CPA calculamos o $r$. Posteriormente fizemos o HMAC sobre este para verificar a correção do algoritmo.

##### Encrypt() 


##### Decrypt() 


  
### NewHope

#### KEM-IND-CPA  
  
  
##### KeyGen()  
 
 - Nesta função começamos por gerar uma $seed$ de tamanho 32 e usando esta $seed$ com o $shake256()$ obtivemos 2 seeds: $publicSeed$ e $noiseSeed$.
 - Com a $publicSeed$ geramos o $a_hat$ e fazendo uso da $noiseSeed$ juntamente com as funções $Sample()$ e $PolyBitRev()$ após trasformarmos os resultados para o domínio NTT geramos o $s_hat$ e o $e_hat$.
 - O $b_hat$ foi gerado multiplicando o $a_hat$ pelo $s_hat$ e somando ao resultado desta multiplicação o $e_hat$.
 - Por fim foram retornados os pares de chaves $pk$ (b_hat,publicSeed) e $sk$ (s_hat).
  
  
##### Encapsulate()  

- Nesta função foi gerado uma $coin$ de tamanho 32 para ser usada com o prefixo $b"\x02"$ para gerar, usado o $shake256()$, o $k$ e uma $coin_prime$.
- O criptograma $cr$ foi gerado passando á função $Encript()$ a $pk$, o $k$ e a $coin_prime$. O $ss$, de tamanho 32, resultou da aplicação do $shake256()$ ao $k$.
- Por fim retornou-se o par $(cr,ss)$.
  
 
##### Decapsulate()  

- Nesta função passamos ao $Decrypt()$ o criptograma e a chave secreta, obtendo o $k_prime$.
- O $k_prime$ obtido foi então submetido á mesma função $shake256()$ para verificar a correção do algoritmo comparando este com o $ss$ gerado no $Encapsulate()$.

 
##### Encrypt() 

- Primeiro foi verificado o comprimento da mensagem, caso não tivesse tamanho 32 esta função retornava um erro. 
- Caso a mensagem tivesse o comprimento certo, abriamos a chave pública obtendo o $b_hat$ e a $publicSeed$.
- Fazendo uso da $publicSeed$ recalculavamos o $a_hat$ e usando a $coin$ mais uma vez auxiliados pelas funções $Sample()$ e $PolyBitRev()$ geramos os parâmetros $s\_prime$, $e\_prime$ e $e\_prime2$. O $t\_hat$ resultou da passagem do $s\_prime$ para o dominio NTT e o $u\_hat$ obteve-se somando o $e\_prime$, no dominio NTT ao resultado da multiplicação do $a\_hat$ pelo $t\_hat$.  
- De seguida realiou-se o Encode da mensagem ao qual chamamos $v$ e por último obtivemos o $v_prime$ somando o $v$ e o $e_prime2$ ao resultado da multiplicação do $b_hat$ pelo $t_hat$.
- Nesta função retornamos o par ($u_hat$, $v_prime$).


##### Decrypt() 

- O primeiro passo nesta função foi abrir o criptograma em ($u_hat$, $v_prime$). 
- Em seguida calculou-se o $mess_poly$ subtraindo ao $v_prime$ a multiplicação do $u_hat$ pelo $s_hat$.
- Posteriormente bastou fazer o Decode do $mess_poly$ e retornar o resultado.




#### PKE-IND-CPA
  
  
##### KeyGen()  
  
  
##### Encapsulate()  
  
  
##### Decapsulate()  

In [60]:
import hashlib
from random import choice, randint


"""
transpõe os coeficientes de "w" para o intervalo -q//2..+q//2
e arredonda-os ao múltiplo de 3 mais próximo
"""
def f(x):
    return ((x/3).round())*3
    
class NTRU_Prime():
    def __init__(self,w):
        self.w=w #único parâmetro que inicializa a classe
        q=24*self.w
        while True:
            if (1+q).is_prime():
                break 
            else:
                q += 1         
        q=q+1
        
        Zx.<x>  = ZZ[]
        Zq.<z>  = PolynomialRing(GF(q))
    
        p = next_prime(2*self.w)
        while True:
            if  Zq(x^p-x-1).is_irreducible():
                break
            else:
                p = next_prime(p+1)
                
        self.p=p
        self.q=q
    
    
    def small_poly(self,p,t=None):
        """
        polinómios cujos coeficientes são -1, 0, 1
        """
        Zx.<x>  = ZZ[]
        if not t:
            return Zx([choice([-1,0,1]) for k in range(p)])
        u = floor(2*(p-1)//t) ; k = randint(0,u) ; l = [0]*p
        while k < p:
            l[k] = choice([-1,1]) ; k += randint(1,u)
        return Zx(l)
    def Hash(self,t): #função para calcular o hash de um objecto
        ww = reduce(lambda x,y: x + y.binary(), t.list() , "")
        return hashlib.sha256(ww.encode('utf-8')).hexdigest()
    
    
    def round_3(self,t):
        Zx.<x>  = ZZ[]
        r = self.q//2
        pol_list = t.list()
        res = [f(lift(p+r) - r) for p in pol_list]
        return Zx(res) 
    
    
    def round_(self,t,n=-1):
        if n==-1:
            n=self.q
        Zx.<x>  = ZZ[]
        """
        input:  polinómio em Gqr ou Z3r
        output: transpõe os coeficientes para o intervalo -n//2..+n//2
        """
        r = n//2
        res_list = []
        pol_list = t.list()
        for p in pol_list:
            res_list.append(lift(p+r) - r)
        return Zx(res_list)
            

    
    def keygen(self):
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        g = self.small_poly(self.p)
        while not R3(g).is_unit():  # enquanto R3(g) não for invertível, geramos novo g.
            g = self.small_poly(self.p)
        f = self.small_poly(self.p,self.w)
        g_inv = R3(g)^(-1)
        self.secret = (f , g_inv)
        self.pk = Rq(g)/Rq(3*f)  # chave pública

        
    def encapsulate(self):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        #geração de um polinómio pequeno
        r = self.small_poly(self.p,self.w)
        key = self.Hash(r)
        C   = self.round_3(Rq(r)*self.pk)
        return (key, C)
    
    
    def decapsulate(self,C):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        # Decode c, obtaining c ∈ R.
        (f , g_inv) = self.secret
        #Multiply by 3f in R/q.
        #View each coefficient of 3fc in R/q as an integer between −(q − 1)/2 and (q − 1)/2,
        # reduce modulo 3, obtaining a polynomial e in R/3
        pre_process = Rq(3*f) * Rq(C)
        e = g_inv * R3(self.round_(pre_process)) ;
        w = self.round_(e,n=3) ;
        key = self.Hash(w)
        return key

In [61]:
A=NTRU_Prime(286)
A.keygen()
(key,C) = A.encapsulate()
key == A.decapsulate(C)





True

In [1]:
import hashlib, hmac
from sage.crypto.util import ascii_integer
from random import choice, randint


"""
transpõe os coeficientes de "w" para o intervalo -q//2..+q//2
e arredonda-os ao múltiplo de 3 mais próximo
"""
def f(x):
    return ((x/3).round())*3
    
"""
concatena 2 polinómios
"""
def merge(a,b):
    s = []
    if len(a) != len(b):
        print("unequal sizes")
        return False
    for i in range(len(a)):
        s.append(a[i]+b[i])
    return s
    
    
def _toZ(f,p=None):
    ff = list(f)
    if p == None:
        return ff
    else:
        fp = map(lift,[Mod(a,p) for a in ff])
        return [u if u <= p//2 else u-p for u in fp ]
    
'''
Util para converter listas de inteiros para uma string que prepresente esses inteiros por ordem correta
'''
def _toStr(intList):
    bin = BinaryStrings()
    s = [str(i) for i in intList]
    return bin.encoding("".join(s))
    

    
def _toBytes(salt):
    bts = [int(s).to_bytes( 1 , byteorder='big', signed=True ) for s in salt]
    res = b''.join(bts)
    return res
    
class NTRU_Prime():
    def __init__(self,w):
        self.w=w #único parâmetro que inicializa a classe
        q=24*self.w
        while True:
            if (1+q).is_prime():
                break 
            else:
                q += 1         
        q=q+1
        
        Zx.<x>  = ZZ[]
        Zq.<z>  = PolynomialRing(GF(q))
    
        p = next_prime(2*self.w)
        while True:
            if  Zq(x^p-x-1).is_irreducible():
                break
            else:
                p = next_prime(p+1)
                
        self.p=p
        self.q=q
        self.salt = []
    

    def msg_gen(self):
        return  [choice([-1,0,1]) for k in range(self.q)]
    
    def small_poly(self,p,t=None):
        """
        polinómios cujos coeficientes são -1, 0, 1
        """
        Zx.<x>  = ZZ[]
        if not t:
            return Zx([choice([-1,0,1]) for k in range(p)])
        u = floor(2*(p-1)//t) ;
        k = randint(0,u);
        l = [0]*p
        while k < p:
            l[k] = choice([-1,1]) ;
            k += randint(1,u)
        return Zx(l)
    
    
    def round_3(self,t):
        Zx.<x>  = ZZ[]
        r = self.q//2
        pol_list = t.list()
        res = [f(lift(p+r) - r) for p in pol_list]
        return Zx(res) 
    
    
    def round_(self,t,n=-1):
        if n==-1:
            n=self.q
        Zx.<x>  = ZZ[]
        """
        input:  polinómio em Gqr ou Z3r
        output: transpõe os coeficientes para o intervalo -n//2..+n//2
        """
        r = n//2
        res_list = []
        pol_list = t.list()
        for p in pol_list:
            res_list.append(lift(p+r) - r)
        return Zx(res_list)
            

    
    def keygen(self):
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        g = self.small_poly(self.p)
        while not R3(g).is_unit():  # enquanto R3(g) não for invertível, geramos novo g.
            g = self.small_poly(self.p)
        f = self.small_poly(self.p,self.w)
        g_inv = R3(g)^(-1)
        self.secret = (f , g_inv)
        self.pk = Rq(g)/Rq(3*f)  # chave pública

        
    def saltGenerator(self,p):
        """
        polinómio cujos coeficientes são 0, 1
        """
        salt = [choice([0,1]) for k in range(p)]
        
        return salt
    

    def encapsulate(self):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        
        #geração de um polinómio pequeno em R/p
        r = _toZ(self.small_poly(self.p,self.w))
        
        #geração de um salt
        self.salt = self.saltGenerator(len(r))
        btarray = _toBytes(self.salt)
        
        #Criptograma
        C   = self.round_3(Rq(r)*self.pk)
        
        #Key
        key = hmac.new(btarray, _toBytes(r), hashlib.sha1).digest()
        return (key, self.encode(C))
    
    def encrypt(self):
        
    
    def decode(self,c):
        return
    
    #recebe o objeto C devolvido pela função Encapsulate 
    def decapsulate(self,c):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        # Descodificação da string para criptograma
        C = self.decode(c)
        # Abrir a chave secreta
        (f , g_inv) = self.secret
        #1ºpasso
        pre_process = Rq(3*f) * Rq(C)
        #2ºpasso -> obtenção do e 
        e = g_inv * R3(self.round_(pre_process)) ;
        #3ºpasso -> obtenção do r
        w = self.round_(e,n=3);
        wStr = _toStr(w)
        btarray = _toBytes(self.salt)
        key = hmac.new(btarray, _toBytes(w), hashlib.sha1).digest()
        
        return key

In [2]:
A=NTRU_Prime(286)

In [3]:
A=NTRU_Prime(286)
A.p,A.q,A.w

(757, 6869, 286)

In [4]:
A.keygen()

In [5]:
(key,C) = A.encrypt()

In [6]:
key2 = A.decrypt(C)
print(key == key2)

True


# NewHope

In [1]:
import os
import array
import math
import hashlib
import pickle

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import (
    Cipher, algorithms, modes
)

class New_Hope_PKE_IND_CPA:
    
    def __init__(self,n=512):
        if not n in [512,1024]:
            raise ValueError("Not Accepted Value of N")
        self.n = n
        
        self.sigma = None
        
        self.q = 1 + 2*n
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
        
        # Defining The Ring Z_q[X]/(X^n + 1)
        self.F = GF(self.q)
        self.R_original = PolynomialRing(self.F,'x')
        R = self.R_original.quotient(x**(self.n) + 1,"a")
        self.R = R
        
        w = (self.R_original).gen();
        self.w = w
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])
        
    
    def b2i(self,number):
        return number
    
    def PolyBitRev(self,poly):
        v_array = [0]*self.n
        for i in range(0,self.n):
            v_array[self.BitRev(i)] = poly[i]
        return self.R(v_array)
    
    def BitRev(self,index):
        result = 0
        for i in range(0,int(math.log(self.n,2))):
            result += (((index >> i) & 1) << (int(math.log(self.n,2)) - 1 - i))
        return result
    
    def hamming_weight(self,n):
        c = 0
        while n:
            c += 1
            n &= n - 1

        return c
    
    def Sample(self,seed,nonce):
        nonce = nonce & 0xFF
        r_array = []
        extseed = [0]*34
        extseed[:32] = seed
        extseed[32] = nonce
        for i in range(0,self.n/64):
            extseed[33] = i
            buf = self.shake256(128,extseed)
            for j in range(0,64):
                a = buf[2*j]
                b = buf[2*j+1]
                r_array.append((self.hamming_weight(a) + self.q - self.hamming_weight(b)) % self.q)
        return self.R(r_array)
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; 
            xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ;
            f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ;
            ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ;
            ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ;
                b = s*ff1[i]  
                ff[i] = a + b ; 
                ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
    
    def ntt_inverse(self,ff):               
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def component_multiplication(self,poly_one,poly_two):
        result_poly = []
        for i in range(0,self.n):
            result_poly.append(poly_one[i] * poly_two[i])
        return self.R(result_poly)
    
    def shake256(self,length,message):
        m = hashlib.shake_256()
        m.update(bytearray(message))
        return m.digest(int(length))
    
    def shake128_Absorb(self,message):
        self.state_number = 0
        m = hashlib.shake_128()
        m.update(bytearray(message))
        return m
    
    def shake128_Squeeze(self,j,state):
        self.state_number+=j
        length_digest = (self.state_number + j) * 168
        return (state.digest(int(length_digest))[(self.state_number*168):],state)
    
    def GenA(self,seed):
        a_array = []
        extseed = [0]*33
        extseed[:32] = seed
        for i in range(0,self.n/64):
            ctr = 0
            extseed[32] = i
            state = self.shake128_Absorb(extseed)
            while ctr < 64:
                (buf,state) = self.shake128_Squeeze(1,state)
                j = 0
                while j < 168 and ctr < 64:
                    val = self.b2i(buf[j]) | (self.b2i(buf[j+1]) << 8)
                    if val < 5*self.q:
                        a_array.append(val)
                        ctr = ctr+1
                    j = j+2
        return self.R(a_array)
    
    def Decode(self,coded_message):
        message = [0]*32 # Initializes an array of 32 bytes
        for i in range(0,256):
            t = abs(int(coded_message[i]) - int((self.q - 1)/2))
            t = t + abs(int(coded_message[i+256]) - int((self.q-1)/2))
            if self.n == 1024:
                t = t + abs(int(coded_message[i+512]) - int((self.q - 1)/2))
                t = t + abs(int(coded_message[i+768]) - int((self.q - 1)/2))
                t = t-int(self.q)
            else:
                t = t - int(self.q/2)
            t = t >> 15
            index = i >> 3
            t = -t
            
            message[index] += (2**(i % 8)) * t
        return bytearray(message)
    
    def Encode(self,message):
        message = bytearray(message)  
        v = [0]*1024
        for i in range(0,32):
            for j in range(0,8):
                mask = -((message[i] >> j) & 1)
            
                v[8*i + j + 0] = int(mask) & (int(self.q/2))
                v[8*i + j + 256] = int(mask) & (int(self.q/2))
                if self.n == 1024:
                    v[8*i + j + 512] = int(mask) & (int(self.q/2))
                    v[8*i + j + 768] = int(mask) & (int(self.q/2))
        
        return self.R(v)
    def Compress(self,poly):
        k = 0
        t = [0] * 8
        q = int(self.q.n())
        
        h = []
        
        for l in range(0,self.n -1):
            i = 8*l
            for j in range(0,8):
                t[j] = poly[i+j] % q
                t[j] = int((((int(t[j]) << 3) + q/2)/q)) & 7
            h.append( t[0] | (t[1] << 3) | (t[2] << 6) )
            h.append( (t[2] >> 2) | (t[3] << 1) | (t[4] << 4) | (t[5] << 7))
            h.append((t[5] >> 1) | (t[6] << 2) | (t[7] << 5))
        return h
    
    def Decompress(self,a):
        k = 0
        r = [0]*self.n
        for l in range(0,self.n/8):
            i = 8*l
            r[i] = a[k + 0] & 7
            r[i + 1] = (a[k + 0] >> 3) & 7
            r[i + 2] = (a[k+ 0] >> 6)|((a[k+ 1] << 2) & 4)
            r[i + 3] = (a[k+ 1] >> 1) & 7
            r[i + 4] = (a[k+ 1] >> 4) & 7
            r[i + 5] = (a[k+ 1] >> 7)|((a[k+ 2] << 1) & 6)
            r[i + 6] = (a[k+ 2] >> 2) & 7
            r[i + 7] = (a[k+ 2] >> 5)
            k = k + 3
            for j in range(0,8):
                r[i+j] = (r[i+j] * int(self.q) + 4) >> 3
        return self.R(r)
    
    def EncodeC(self,c):
        (u_hat,h) = c
        r = []
        r[0:((7*self.n/4))] = self.EncodePolynomial(u_hat)
        r[(7*self.n/4):] = h
        return r
    
    def DecodeC(self,enc):
        u_hat = self.DecodePolynomial(enc[0:((7*self.n/4))])
        h = enc[(7*self.n/4):]
        return (u_hat,h)
    
    def EncodePolynomial(self,s_hat):
        r = []
        for i in range(0,self.n/4):
            t0 = int(s_hat[4*i + 0])
            t1 = int(s_hat[4*i + 1])
            t2 = int(s_hat[4*i + 2])
            t3 = int(s_hat[4*i + 3])
            r.append(t0&0xff)
            r.append((t0 >> 8)|(t1 << 6)&0xff)
            r.append((t1 >> 2)&0xff)
            r.append((t1 >> 10)|(t2 << 4)&0xff)
            r.append((t2 >> 4)&0xff)
            r.append((t2 >> 12)|(t3 << 2)&0xff)
            r.append((t3 >> 6)&0xff)
        return r
    
    def DecodePolynomial(self,v):
        r = []
        for i in range(0,self.n/4):
            r.append(int(v[7*i + 0])|(int(v[7*i+ 1]) & 0x3f) << 8)
            r.append((int(v[7*i+ 1])>>6)|(int(v[7*i+ 2]) <<2)|((int(v[7*i+ 3]) & 0x0f) <<10))
            r.append((int(v[7*i+ 3]) >> 4)|(int(v[7*i+ 4]) << 4)|((int(v[7*i+ 5]) & 0x03) << 12))
            r.append((int(v[7*i+ 5]) >> 2)|(int(v[7*i+ 6]) << 6))
        return r
    
    def EncodePK(self,b,publicSeed):
        r = []
        r[:((7*self.n/4))] = self.EncodePolynomial(b)
        r[7*self.n/4:] = publicSeed
        return r
    
    def DecodePK(self,pk):
        b = self.DecodePolynomial(pk[:(7*self.n/4)])
        seed = pk[(7*self.n/4):]
        return (b,seed)
    
    def KeyGen(self):
        seed = os.urandom(32)
        z = self.shake256(64,(int(0x01)).to_bytes(1,byteorder="big") + seed)
        publicSeed = z[:32]
        noiseSeed = z[32:]
        a_hat = self.GenA(publicSeed)
        s = self.PolyBitRev(self.Sample(noiseSeed,0))
        s_hat = self.R(self.ntt(s))
        e = self.PolyBitRev(self.Sample(noiseSeed,1))
        e_hat = self.R(self.ntt(e))
        b_hat = self.component_multiplication(a_hat,s_hat) + e_hat
        return(self.EncodePK(b_hat,publicSeed),self.EncodePolynomial(s_hat))
    
    def Encrypt(self,pk,message,coin):
        # PK 7*n/4 + 32
        # mess 32
        # coin 32
        if len(message) != 32:
            raise ValueError("Message must 32 bytes long")
        (b_hat,publicSeed) = self.DecodePK(pk)
        a_hat = self.GenA(publicSeed)
        s_prime = self.PolyBitRev(self.Sample(coin,0))
        e_prime = self.PolyBitRev(self.Sample(coin,1))
        e_prime2 = self.Sample(coin,2)
        t_hat = self.ntt(s_prime)
        u_hat = self.component_multiplication(a_hat,t_hat) + self.R(self.ntt(e_prime))
        v = self.Encode(message)
        v_prime = self.ntt_inverse(self.component_multiplication(b_hat,t_hat)) + e_prime2 + v
        h = self.Compress(v_prime)
        return self.EncodeC((u_hat,h))
    
    def Decrypt(self,cripto,sk):
        (u_hat,h) = self.DecodeC(cripto)
        s_hat = self.DecodePolynomial(sk)
        v_prime = self.Decompress(h)
        mess_poly = v_prime - self.ntt_inverse(self.component_multiplication(u_hat,s_hat))
        mess = self.Decode(mess_poly)
        return mess

class New_Hope_KEM_IND_CPA:
    
    def __init__(self,n=512):
        self.pke = New_Hope_PKE_IND_CPA(n)
    
    def KeyGen(self):
        return self.pke.KeyGen()
    
    def Encapsulate(self,pk):
        coin = os.urandom(32)
        result = self.pke.shake256(64,b"\x02" + coin)
        K = result[:32]
        coin_prime = result[32:]
        cr = self.pke.Encrypt(pk,K,coin_prime)
        ss = self.pke.shake256(32,K)
        return (cr,ss)
    
    def Decapsulate(self,c,sk):
        K_prime = self.pke.Decrypt(c,sk)
        return self.pke.shake256(32,K_prime)

class New_Hope_KEM_IND_CCA():
    
    def __init__(self,n=512):
        self.n = n
        self.pke = New_Hope_PKE_IND_CPA(n)
        
    def KeyGen(self):
        (pk,sk) = self.pke.KeyGen()
        s = os.urandom(32)
        
        return (pk, bytearray(sk) + bytearray(pk) + self.pke.shake256(32,pk) + s)
    
    def Encapsulate(self,pk):
        coin = os.urandom(32)
        miu = self.pke.shake256(32,b"\x04" + coin)
        result = self.pke.shake256(96,b"\x08" + miu + self.pke.shake256(32,pk))
        K = result[:32]
        coin_prime = result[32:64]
        d = result[64:]
        c = self.pke.Encrypt(pk,miu,coin_prime)

        ss = self.pke.shake256(32,K + self.pke.shake256(32,pickle.dumps(c) + d))
 
        return (pickle.dumps(c) + d, ss)
    
    def Decapsulate(self,c_bar,sk_bar):
        c = pickle.loads(c_bar[:-32])
        d = c_bar[-32:]

        sk = sk_bar[:7*self.n/4]
        pk = sk_bar[7*self.n/4:7*self.n/2 + 32]
        h = sk_bar[7*self.n/2 + 32:7*self.n/2 + 64]
        s = sk_bar[32:7*self.n/2 + 64:]
        
        miu = self.pke.Decrypt(c,sk)
        
        result = self.pke.shake256(96,b"\x08" + miu + h)
        K_prime = result[:32]
        coin_prime2 = result[32:64]
        d_prime = result[64:]
        fail = 1
        K = s
        if c == self.pke.Encrypt(pk,miu,coin_prime2) and d == d_prime:
            fail = 0
            K = K_prime
        return self.pke.shake256(32,K + self.pke.shake256(32,pickle.dumps(c) + d))
        
class New_Hope_PKE_IND_CCA:
    
    def __init__(self,n=512):
        self.n = n
        self.kem = New_Hope_KEM_IND_CCA(n)
    
    def KeyGen(self):
        return self.kem.KeyGen()
    
    def Encrypt(self,pk,message):
        (enc,ss) = self.kem.Encapsulate(pk)
        iv = os.urandom(12)
        encryptor = Cipher(algorithms.AES(ss),modes.GCM(iv),backend=default_backend()).encryptor()
        ciphertext = encryptor.update(message) + encryptor.finalize()
        return (enc,ciphertext,encryptor.tag,iv)
    
    def Decrypt(self,enc,sk,tag,iv,message):
        ss = self.kem.Decapsulate(enc,sk)
        decryptor = Cipher(algorithms.AES(ss),modes.GCM(iv,tag),backend=default_backend()).decryptor()
        return decryptor.update(message) + decryptor.finalize()
        
pke = New_Hope_PKE_IND_CPA()

(pk,sk) = pke.KeyGen()
message = b"3a3a22t2c22f226q22b2hlop22222222"
cripto = pke.Encrypt(pk,message,os.urandom(32))
print("PKE-IND-CPA")
print(message)
print(pke.Decrypt(cripto,sk))
print("\n")
kem = New_Hope_KEM_IND_CPA()
(pk,sk) = kem.KeyGen()
(c,ss) = kem.Encapsulate(pk)
rss = kem.Decapsulate(c,sk)
print("KEM-IND-CPA")
print(ss)
print(rss)
print("\n")

print("KEM_IND_CCA")
kem_cca = New_Hope_KEM_IND_CCA()
(pk,sk) = kem_cca.KeyGen()
(enc,ss) = kem_cca.Encapsulate(pk)
rss = kem_cca.Decapsulate(enc,sk)
print(ss)
print(rss)

print("\n")

print("PKE_IND_CCA")
pke_cca = New_Hope_PKE_IND_CCA()
(pk,sk) = pke_cca.KeyGen()
message = b"aksdjalkfjf"
print(message)
(enc,cipher,tag,iv) = pke_cca.Encrypt(pk,message)
print(pke_cca.Decrypt(enc,sk,tag,iv,cipher))
print("\n")

PKE-IND-CPA
b'3a3a22t2c22f226q22b2hlop22222222'
bytearray(b'3a3a22t2c22f226q22b2hlop22222222')


KEM-IND-CPA
b'\xd6\xa3\xb7C\xa7\xaa\x99n\xb1w\xe3\xdf\x9f\x06\xaf\x17\x9a\x8a\x16\x91\xd0\x10(A\xf7q7\x11\x8b\x0bU\x85'
b'\xd6\xa3\xb7C\xa7\xaa\x99n\xb1w\xe3\xdf\x9f\x06\xaf\x17\x9a\x8a\x16\x91\xd0\x10(A\xf7q7\x11\x8b\x0bU\x85'


KEM_IND_CCA
b'k\x94\xfe\x82vR\x05O\xa3\xc9`\x06!\x1ea\x18\x87\x1a\x8a\xe2I\x9d#`ufMs3\xd9=\xda'
b'k\x94\xfe\x82vR\x05O\xa3\xc9`\x06!\x1ea\x18\x87\x1a\x8a\xe2I\x9d#`ufMs3\xd9=\xda'


PKE_IND_CCA
b'aksdjalkfjf'
b'aksdjalkfjf'


